## Расчет похожести товаров по тестовому описанию

In [1]:
import pandas as pd
import numpy as np
from string import punctuation
from pymystem3 import Mystem
import re

import nltk
from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

import pymorphy2

In [2]:
!pip install pymorphy2
!pip install pymystem3
!pip install nltk

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/Alena/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Alena/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# cd /content/gdrive/My Drive/python

/content/gdrive/My Drive/python


## Считывание общих данных , тестовых и тренировочных данных

In [3]:
data = pd.read_csv('data/details.csv')

In [4]:
data.head()

,itemid,attribute_name,attribute_value
0,90672,Name,Одиночка - Джек
1,90672,Annotation,"Аллан, герой романа `Новичок`, не был преступн..."
2,100658,Name,Начала численного анализа
3,100658,Annotation,"Написана по материалом лекций, которые, один ..."
4,120997,Name,Физика процессов эволюции


In [13]:
data['attribute_name'] = data['attribute_name'].map({'Name': 0, 'Annotation': 1}).astype('bool')
data.head()

,itemid,attribute_name,attribute_value
0,90672,False,одиночка джек
1,90672,True,аллан герой роман новичок преступник знать пут...
2,100658,False,начало численный анализ
3,100658,True,написать материал лекция который автор локуцие...
4,120997,False,физика процесс эволюция


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260590 entries, 0 to 1260589
Data columns (total 3 columns):
itemid             1260590 non-null int64
attribute_name     1260590 non-null bool
attribute_value    1260589 non-null object
dtypes: bool(1), int64(1), object(1)
memory usage: 20.4+ MB


In [23]:
train_data = pd.read_csv("data/train_with_scores.csv")
train_data['clientid'] = train_data['clientid'].astype('uint32')
train_data['jointitemid'] = train_data['jointitemid'].astype('uint32')
train_data['label']=train_data['label'].astype('int8')
train_data['novelty_cnt']=train_data['novelty_cnt'].astype('int32')
train_data['count_on_session_view']=train_data['count_on_session_view'].astype('int32')
train_data['count_on_session_to_cart']=train_data['count_on_session_to_cart'].astype('int32')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1390438 entries, 0 to 1390437
Data columns (total 19 columns):
clientid                          1390438 non-null uint32
itemid                            1390438 non-null int64
jointitemid                       1390438 non-null uint32
label                             1390438 non-null int8
timestamp                         1390438 non-null object
view_cnt                          1390438 non-null float64
to_cart_cnt                       1390438 non-null float64
ctr                               1390438 non-null float64
novelty_cnt                       1390438 non-null int32
to_cart_day_avg_cnt               1390438 non-null float64
view_day_avg_cnt                  1390438 non-null float64
to_cart_Lust_Day_cnt              1390438 non-null float64
last_day_views                    1390438 non-null float64
lastDayToCart/dayAvgToCart_cnt    1390438 non-null float64
lastDayView/dayAvgView_cnt        1390438 non-null float64
same_items_o

In [24]:
test_data = pd.read_csv("data/test_with_scores.csv")
test_data['clientid'] = test_data['clientid'].astype('uint32')
test_data['jointitemid'] = test_data['jointitemid'].astype('uint32')
test_data['label'] = test_data['label'].astype('int8')
test_data['novelty_cnt'] = test_data['novelty_cnt'].astype('int32')
test_data['count_on_session_view'] = test_data['count_on_session_view'].astype('int32')
test_data['count_on_session_to_cart'] = test_data['count_on_session_to_cart'].astype('int32')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597158 entries, 0 to 597157
Data columns (total 19 columns):
clientid                          597158 non-null uint32
itemid                            597158 non-null int64
jointitemid                       597158 non-null uint32
label                             597158 non-null int8
timestamp                         597158 non-null object
view_cnt                          597158 non-null float64
to_cart_cnt                       597158 non-null float64
ctr                               597158 non-null float64
novelty_cnt                       597158 non-null int32
to_cart_day_avg_cnt               597158 non-null float64
view_day_avg_cnt                  597158 non-null float64
to_cart_Lust_Day_cnt              597158 non-null float64
last_day_views                    597158 non-null float64
lastDayToCart/dayAvgToCart_cnt    597158 non-null float64
lastDayView/dayAvgView_cnt        597158 non-null float64
same_items_on_session_view   

## Токенизация, нормализация тескста

In [8]:
#использование Mystem
def Lemmatisation(text_col,stopwords):
    lemmatize_func = Mystem().lemmatize
    
    pattern = r'\b(?:{})\b'.format('|'.join(stopwords))
    lem_text = (
        text_col
        .str.lower()
        .str.replace(r"<[^>]+>|[^a-zа-яё0-9]", " ")
        .str.replace(r"(\s)", " ")
        .str.strip() # удаление пробелов в начале и в конце
        .apply(lambda x: lemmatize_func(x) if isinstance(x, str) else None) # лемматизация
        .apply(lambda x: ' '.join(x) if isinstance(x, list) else None) # соединение лемматизированных слов
        .str.replace(pattern, '') # удаление стоп-слов
        .str.replace(r'\b(\w)\b', '') # удаление слов из одной буквы
        .str.replace(r'\s+', ' ') # любое количество пробелов на 1 пробел
        .str.strip() # удаление пробелов в начале и в конце
    )
    return  lem_text


In [9]:
%%time
data['attribute_value'] = Lemmatisation(data['attribute_value'],stopwords.words('russian'))

CPU times: user 8min 40s, sys: 3min 51s, total: 12min 32s
Wall time: 1h 13min 38s


## Разбиение данных по аннотациям и названиям

In [25]:
#переиндиксация товаров
def newIndex(data):
    items = data['itemid'].unique()
    items_cat = np.arange(0, len(items), dtype='uint32')
    items_mapping = pd.DataFrame({'old': items, 'new': items_cat})
    data['itemid'] = data['itemid'].map(items_mapping.set_index('old').new)
    return (data, items_mapping)

### аннотации

In [26]:
data_annotation = data[data['attribute_name']].reset_index(drop = True)
data_annotation['itemid'], items_mapping_annotation = newIndex(data_annotation)
data_annotation.head()

,itemid,attribute_name,attribute_value
0,0,True,аллан герой роман новичок преступник знать пут...
1,1,True,написать материал лекция который автор локуцие...
2,2,True,книга известный немецкий физик эбелинг ученик ...
3,3,True,издание являться учебный пособие новый предмет...
4,4,True,третий собрание сочинение включать произведени...


In [27]:
data_annotation['attribute_value'] = data_annotation['attribute_value'].apply(lambda x: np.str_(x))

In [17]:
items_mapping_annotation.head()

,old,new
0,90672,0
1,100658,1
2,120997,2
3,933272,3
4,971258,4


### названия

In [28]:
data_name = data[~data['attribute_name']].reset_index(drop = True)
data_name['itemid'], items_mapping_name = newIndex(data_name)
data_name.head()

,itemid,attribute_name,attribute_value
0,0,False,одиночка джек
1,1,False,начало численный анализ
2,2,False,физика процесс эволюция
3,3,False,урок госпожа мелодия учебник класс детский муз...
4,4,False,эрнест хемингуэй собрание сочинение четыре


In [29]:
#items_mapping_name.head()
data_name[['attribute_value']]

,attribute_value
0,одиночка джек
1,начало численный анализ
2,физика процесс эволюция
3,урок госпожа мелодия учебник класс детский муз...
4,эрнест хемингуэй собрание сочинение четыре
5,начало
6,трипланетие первый ленсман
7,младший современник шекспир
8,западный восточный диван
9,жизнеописание чжа юаньчжан


## Составление матирцы tf-idf

In [30]:
#Матрица для имен
tfidf_vectorizer = TfidfVectorizer(norm='l2')
tf_idf_matrix_name = tfidf_vectorizer.fit_transform(data_name['attribute_value'])
print(tf_idf_matrix_name.shape)

(653916, 261924)


In [31]:
#Матрица для аннотаций
tfidf_vectorizer = TfidfVectorizer(norm='l2')
tf_idf_matrix_annotation = tfidf_vectorizer.fit_transform(data_annotation['attribute_value'])
print(tf_idf_matrix_annotation.shape)

(606674, 349499)


Считаем косинусную похожесть товаров


In [32]:
def count_same_score(data, matrix, attribute_name):
    pairs = data.dropna()[['item_cat', 'jointitem_cat']]
    pairs['item_cat'] = pairs['item_cat'].astype('uint32')
    pairs['jointitem_cat'] = pairs['jointitem_cat'].astype('uint32')

    pairs['same_' +attribute_name] = (matrix[pairs['item_cat'].values]
                                     .multiply(matrix[ pairs['jointitem_cat'].values]).sum(axis=1) )
    data = pd.merge(data, pairs, on = ['item_cat', 'jointitem_cat'], how = 'left').drop_duplicates()
    return data
  

# предобработка файлов теста и трейна
## запись признака похожести описаний и названий товаров

## target_train

### Для названий

In [33]:
train_data['item_cat'] = train_data['itemid'].map(items_mapping_name.set_index('old').new)
train_data['jointitem_cat'] =train_data['jointitemid'].map(items_mapping_name.set_index('old').new)

In [34]:
train_data = count_same_score(train_data, tf_idf_matrix_name, 'name')
train_data[train_data['same_name'] > 0].head()

,clientid,itemid,jointitemid,label,timestamp,view_cnt,to_cart_cnt,ctr,novelty_cnt,to_cart_day_avg_cnt,...,last_day_views,lastDayToCart/dayAvgToCart_cnt,lastDayView/dayAvgView_cnt,same_items_on_session_view,count_on_session_view,same_items_on_session_to_cart,count_on_session_to_cart,item_cat,jointitem_cat,same_name
2,19548158,147389610,148381589,0,2019-08-31 22:32:31,24.0,9.0,2.666667,737272,1.125000,...,0.0,0.000,0.000000,0.160623,5,0.308607,2,61818.0,469989.0,0.093962
15,17627524,149313915,149295846,0,2019-09-03 18:58:54,14.0,3.0,4.666667,737275,1.000000,...,0.0,0.000,0.000000,0.154713,3,0.000000,0,633183.0,404345.0,0.018278
41,37917745,150533609,150533637,0,2019-09-06 21:54:36,3.0,4.0,0.750000,737277,1.000000,...,0.0,0.000,0.000000,0.000000,0,0.000000,0,289882.0,626050.0,0.014222
165,9643075,154504149,153480041,0,2019-09-03 09:21:19,6.0,1.0,6.000000,737275,1.000000,...,0.0,0.000,0.000000,0.000000,0,0.000000,0,438146.0,473402.0,0.017880
185,1801423,144295520,138896975,1,2019-09-04 18:42:12,128.0,80.0,1.600000,737272,3.076923,...,3.0,0.975,0.679688,0.000000,0,0.000000,0,160205.0,280919.0,0.025703


### Для аннотаций

In [35]:
train_data['item_cat'] = train_data['itemid'].map(items_mapping_annotation.set_index('old').new)
train_data['jointitem_cat'] = train_data['jointitemid'].map(items_mapping_annotation.set_index('old').new)

In [36]:
train_data = count_same_score(train_data, tf_idf_matrix_annotation, 'annotation')
train_data.head()

,clientid,itemid,jointitemid,label,timestamp,view_cnt,to_cart_cnt,ctr,novelty_cnt,to_cart_day_avg_cnt,...,lastDayToCart/dayAvgToCart_cnt,lastDayView/dayAvgView_cnt,same_items_on_session_view,count_on_session_view,same_items_on_session_to_cart,count_on_session_to_cart,item_cat,jointitem_cat,same_name,same_annotation
0,7833842,31499843,138176581,1,2019-09-07 20:11:01,31.0,9.0,3.444444,737272,1.500,...,0.0,0.0,0.000000,0,0.069338,1,227433.0,487143.0,0.000000,0.009087
2,19548158,147389610,148381589,0,2019-08-31 22:32:31,24.0,9.0,2.666667,737272,1.125,...,0.0,0.0,0.160623,5,0.308607,2,57521.0,NaN,0.093962,NaN
3,32943407,6261257,4490956,0,2019-09-06 15:19:30,0.0,1.0,0.000000,0,1.000,...,0.0,0.0,0.000000,0,0.000000,0,222530.0,364685.0,0.000000,0.338115
4,10185243,148455169,148455173,0,2019-09-03 17:43:50,10.0,1.0,10.000000,737275,1.000,...,0.0,0.0,0.000000,0,0.000000,0,299719.0,226173.0,0.000000,0.456733
5,30552232,152440009,152440052,0,2019-09-01 07:07:59,9.0,1.0,9.000000,737274,1.000,...,0.0,0.0,0.005222,1,0.000000,0,94682.0,157365.0,0.000000,0.485534


In [37]:
train_data = train_data.fillna(0)
train_data = train_data.drop(columns=["item_cat","jointitem_cat"])

## target_test

### Для названий

In [38]:
test_data['item_cat'] = test_data['itemid'].map(items_mapping_name.set_index('old').new)
test_data['jointitem_cat'] = test_data['jointitemid'].map(items_mapping_name.set_index('old').new)

In [39]:
test_data = count_same_score(test_data, tf_idf_matrix_name, 'name')
test_data[test_data['same_name'] > 0].head()

,clientid,itemid,jointitemid,label,timestamp,view_cnt,to_cart_cnt,ctr,novelty_cnt,to_cart_day_avg_cnt,...,last_day_views,lastDayToCart/dayAvgToCart_cnt,lastDayView/dayAvgView_cnt,same_items_on_session_view,count_on_session_view,same_items_on_session_to_cart,count_on_session_to_cart,item_cat,jointitem_cat,same_name
20,28549744,154701785,148837776,0,2019-09-07 18:05:23,17.0,0.0,0.000000,737272,0.000000,...,0.0,0.000000,0.000000,0.000000,0,0.00000,0,120245.0,340583.0,0.043885
29,13019718,149096258,149096269,0,2019-09-05 04:54:40,56.0,4.0,14.000000,737272,1.000000,...,1.0,1.000000,0.410714,0.060302,1,0.00000,0,142702.0,525749.0,0.077509
68,4359122,152256060,152256031,0,2019-09-01 16:47:16,33.0,1.0,33.000000,737272,1.000000,...,0.0,0.000000,0.000000,0.000000,0,0.00000,0,450675.0,36647.0,0.026388
69,10843971,153981139,138334404,0,2019-09-02 10:33:56,72.0,7.0,10.285714,737273,1.166667,...,0.0,0.000000,0.000000,0.037398,1,0.00000,0,604036.0,454022.0,0.064791
119,37942867,152440026,152440009,0,2019-09-07 00:48:23,4155.0,306.0,13.578431,737271,9.562500,...,6.0,0.627451,0.046209,0.115834,257,0.06892,12,313224.0,101780.0,0.120693


### Для аннотаций

In [40]:
test_data['item_cat'] = test_data['itemid'].map(items_mapping_annotation.set_index('old').new)
test_data['jointitem_cat'] = test_data['jointitemid'].map(items_mapping_annotation.set_index('old').new)

In [41]:
test_data= count_same_score(test_data, tf_idf_matrix_annotation, 'annotation')
test_data.head()

,clientid,itemid,jointitemid,label,timestamp,view_cnt,to_cart_cnt,ctr,novelty_cnt,to_cart_day_avg_cnt,...,lastDayToCart/dayAvgToCart_cnt,lastDayView/dayAvgView_cnt,same_items_on_session_view,count_on_session_view,same_items_on_session_to_cart,count_on_session_to_cart,item_cat,jointitem_cat,same_name,same_annotation
0,8081929,152898248,152875664,0,2019-09-06 18:57:23,10.0,2.0,5.000000,737276,1.000000,...,0.000000,0.000000,0.000000,0,0.000000,0,116503.0,397316.0,0.0,0.232233
1,33378638,144847078,140715321,0,2019-09-02 07:09:31,211.0,19.0,11.105263,737272,1.727273,...,0.578947,0.127962,0.023322,7,0.000000,0,393779.0,206369.0,0.0,0.163595
2,36237195,149717596,149390783,0,2019-09-01 07:14:01,282.0,20.0,14.100000,737272,1.666667,...,0.600000,0.109929,0.247489,61,0.139573,3,434342.0,130368.0,0.0,0.157659
4,4190203,141835810,147130456,0,2019-09-06 07:18:33,892.0,210.0,4.247619,737272,6.774194,...,0.295238,0.069507,0.000000,0,0.011117,1,445192.0,289963.0,0.0,0.071880
5,35362099,24560937,155457711,0,2019-09-05 17:16:28,0.0,2.0,0.000000,0,1.000000,...,0.000000,0.000000,0.000000,0,0.000000,0,188974.0,NaN,0.0,NaN


In [42]:
test_data = test_data.fillna(0)
test_data = test_data.drop(columns=["item_cat","jointitem_cat"])

## Запись в файл 

In [22]:
data.to_csv("data/details_ver_2.csv", index=False)

In [43]:
test_data.to_csv("data/test_with_scores.csv", index=False)
train_data.to_csv("data/train_with_scores.csv", index=False)

In [44]:
train_data.to_csv("data/train_with_scores.csv", index=False)